In [14]:
from langgraph.graph import StateGraph,START,END

from dotenv import load_dotenv

from typing import TypedDict,List,Annotated,Literal

from langchain_openai import ChatOpenAI

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate

from langgraph.graph.message import add_messages

from langchain.schema import AIMessage,HumanMessage

from langchain_core.tools import tool

from langgraph.prebuilt import ToolNode, tools_condition

from langchain.prompts import ChatPromptTemplate

from langchain_core.output_parsers import JsonOutputParser

from data_fetcher import run_select_query

import os

In [15]:
TABLE_SCHEMA = """
You have access to the following database schema:

Table: banking_details
- webtop (string)
- account_number (string)
- account_type (string)
- balance (float)
"""


system_message = (
    "You are an AI that only returns SQL SELECT queries based on user requests.\n"
    "Only return the query in the following JSON format:\n"
    '{{"query": "<SQL query>"}}\n\n'
    "Use only the given schema. Do not guess column or table names.\n"
    f"{TABLE_SCHEMA}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "{input}")
])

In [16]:
load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, api_key=openai_key)

parser = JsonOutputParser()

chain = prompt | llm | parser

class dataobject(TypedDict):
    
    messages : Annotated[List,add_messages]

In [19]:
result = chain.invoke({'input':'give me details about webtop 356'})

query = result['query']

print(query)

SELECT * FROM banking_details WHERE webtop = '356'


In [ ]:
run_select_query(query)